## Import

In [1]:
# 주요 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb
import optuna

# 경고 무시 설정
import warnings
warnings.filterwarnings("ignore")

In [2]:
## 데이터 불러오기
train = pd.read_csv('/Users/jian_lee/Desktop/aiffel/data/House Price Prediction/train.csv')
test = pd.read_csv('/Users/jian_lee/Desktop/aiffel/data/House Price Prediction/test.csv')

## Data Preprocessing

In [3]:
# 테스트 데이터의 'id' 값을 따로 저장합니다.
# 제출 파일을 생성할 때 이 값을 사용합니다.
sub_id = test['id']

# train 데이터에 'is_train' 열을 추가하여 학습 데이터임을 표시하고,
# test 데이터에 'is_train' 열을 추가하여 테스트 데이터임을 표시합니다.
train['is_train'] = 1
test['is_train'] = 0

# 타겟 값(종속 변수)을 분리하여 y 변수에 저장합니다.
# 로그 변환을 적용하여 값의 스케일을 줄이고 모델의 안정성을 높입니다.
y = np.log1p(train['price'])

# 타겟 값(price)을 분리했으므로 train 데이터에서 해당 열을 제거합니다.
train.drop(['price'], axis=1, inplace=True)

# train 데이터와 test 데이터를 합쳐 full_data라는 하나의 데이터프레임으로 만듭니다.
# 동일한 전처리를 적용하기 위함입니다.
full_data = pd.concat([train, test], axis=0)

# 날짜(date) 변수에서 연도와 월 정보만 추출하고 정수형으로 변환합니다.
# 예를 들어, '20141013' -> 201410 형식으로 변환됩니다.
full_data['date'] = full_data['date'].apply(lambda x: int(x[:6]))

# 리모델링 여부 변수를 생성합니다.
# yr_renovated 값이 0이면 리모델링이 없었음을 의미하고, 0이 아니면 리모델링이 있었음을 의미합니다.
# is_renovated 변수는 리모델링 여부를 0 또는 1로 나타냅니다.
full_data['is_renovated'] = full_data['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)

# ID 변수와 yr_renovated 변수를 제거합니다.
# ID 변수는 예측에 불필요하고, yr_renovated 정보는 is_renovated 변수로 대체되었기 때문입니다.
full_data.drop(['id', 'yr_renovated'], axis=1, inplace=True)

# 범주형 변수인 zipcode를 원-핫 인코딩으로 처리합니다.
# 첫 번째 카테고리를 제거(drop_first=True)하여 다중공선성 문제를 방지합니다.
full_data = pd.get_dummies(full_data, columns=['zipcode'], drop_first=True)

# 다시 train과 test 데이터로 분리합니다.
# is_train 값이 1인 데이터는 원래 train 데이터였고, 0인 데이터는 원래 test 데이터입니다.
train = full_data[full_data['is_train'] == 1].drop(['is_train'], axis=1)
test = full_data[full_data['is_train'] == 0].drop(['is_train'], axis=1)

# 데이터 스케일링을 수행합니다.
# StandardScaler를 사용하여 각 특성의 평균을 0, 표준편차를 1로 변환합니다.
scaler = StandardScaler()

# train 데이터에 대해 fit_transform()을 사용하여 스케일링을 적용하고,
# test 데이터에 대해서는 transform()을 사용하여 동일한 변환을 적용합니다.
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

## 모델링

In [4]:
# XGBoost 하이퍼파라미터 튜닝 함수 정의 (Optuna 사용)
# - Optuna 라이브러리를 사용하여 XGBoost 모델의 하이퍼파라미터를 자동으로 탐색합니다.
# - 하이퍼파라미터 최적화를 통해 모델의 예측 성능을 향상시킵니다.

def objective(trial):
    # Optuna가 탐색할 하이퍼파라미터 공간을 정의합니다.
    # trial 객체를 통해 각 하이퍼파라미터 값을 탐색합니다.
    params = {
        'objective': 'reg:squarederror',  # 회귀 문제에 적합한 목표 함수 설정
        'eval_metric': 'rmse',            # 평가 지표로 RMSE(Root Mean Squared Error) 사용
        'booster': 'gbtree',              # 트리 기반 모델 사용
        'max_depth': trial.suggest_int('max_depth', 5, 15),  # 트리의 최대 깊이 (5~15 사이 탐색)
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),  # 학습률 (0.001~0.1 사이 로그 균등 분포)
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # 훈련 데이터 샘플링 비율 (50%~100%)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # 각 트리에서 사용할 특성 비율 (50%~100%)
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),  # L2 정규화 항 가중치 (0~10)
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),  # L1 정규화 항 가중치 (0~10)
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000, step=100),  # 트리의 개수 (1000~2000, 100씩 증가)
        'random_state': 42  # 재현 가능성을 위한 랜덤 시드 설정
    }

    # XGBoost 회귀 모델을 정의하고 하이퍼파라미터를 설정합니다.
    model = xgb.XGBRegressor(**params)

    # 교차 검증을 통해 모델의 성능을 평가합니다.
    # 5-Fold 교차 검증을 사용하고, RMSLE(Root Mean Squared Logarithmic Error)를 평가 지표로 사용합니다.
    scores = cross_val_score(model, train_scaled, y, cv=5, scoring='neg_mean_squared_log_error')

    # 평균 RMSLE 값을 계산합니다.
    # cross_val_score 함수는 음수 값을 반환하므로, 다시 양수로 변환한 후 RMSLE 값을 계산합니다.
    rmsle = np.mean(-scores) ** 0.5

    # Optuna가 최소화해야 할 RMSLE 값을 반환합니다.
    return rmsle

# Optuna로 하이퍼파라미터 튜닝 수행
# - Optuna의 study 객체를 생성하고, 최적화를 수행합니다.
# - direction='minimize'는 최소화할 값을 찾는 것을 의미합니다 (여기서는 RMSLE 최소화).
study = optuna.create_study(direction='minimize')

# objective 함수에서 정의한 하이퍼파라미터 공간을 탐색하며, 총 30회의 실험을 수행합니다.
study.optimize(objective, n_trials=30)

# 최적의 하이퍼파라미터를 출력합니다.
print("Best Hyperparameters:", study.best_params)

[I 2025-01-13 16:28:08,311] A new study created in memory with name: no-name-d525cb55-f65f-46be-a47a-266d3b4fada8
[I 2025-01-13 16:28:18,464] Trial 0 finished with value: 0.012067868708665186 and parameters: {'max_depth': 6, 'learning_rate': 0.00886625274670392, 'subsample': 0.6181908579108373, 'colsample_bytree': 0.5257656203498242, 'reg_lambda': 3.677638283641982, 'reg_alpha': 8.88273093380487, 'n_estimators': 2000}. Best is trial 0 with value: 0.012067868708665186.
[I 2025-01-13 16:28:35,394] Trial 1 finished with value: 0.012232180343993624 and parameters: {'max_depth': 11, 'learning_rate': 0.004793051018001833, 'subsample': 0.7432219050955746, 'colsample_bytree': 0.8852593664811277, 'reg_lambda': 3.7229401303785314, 'reg_alpha': 9.582133352812649, 'n_estimators': 1700}. Best is trial 0 with value: 0.012067868708665186.
[I 2025-01-13 16:28:44,119] Trial 2 finished with value: 0.01165191023519652 and parameters: {'max_depth': 7, 'learning_rate': 0.05332330258341229, 'subsample': 0.5

Best Hyperparameters: {'max_depth': 6, 'learning_rate': 0.014459361612479469, 'subsample': 0.7918472898294175, 'colsample_bytree': 0.5676122726827514, 'reg_lambda': 2.7487424163985423, 'reg_alpha': 0.24290153590571606, 'n_estimators': 1600}


In [5]:
# 최적의 하이퍼파라미터로 모델 학습
# - Optuna에서 탐색한 최적의 하이퍼파라미터(best_params)를 사용하여 최종 XGBoost 모델을 생성합니다.
# - 생성된 모델을 학습 데이터(train_scaled)와 타겟 값(y)을 사용해 학습(fit)합니다.

# Optuna의 최적 하이퍼파라미터(best_params)를 가져옵니다.
best_params = study.best_params

# 최적 하이퍼파라미터를 사용하여 XGBoost 회귀 모델을 생성합니다.
# best_params는 딕셔너리 형태로 전달되며, 각 키-값 쌍이 모델의 하이퍼파라미터로 설정됩니다.
best_model = xgb.XGBRegressor(**best_params)

# train_scaled: 스케일링된 학습 데이터
# y: 로그 변환된 타겟 값
# 최적의 하이퍼파라미터로 설정된 모델을 학습 데이터에 맞춰 훈련합니다.
best_model.fit(train_scaled, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5676122726827514, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.014459361612479469, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1600, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [6]:
# 최종 예측 수행
y_test_pred = np.expm1(best_model.predict(test_scaled))

## Submission

In [7]:
# 제출 파일 생성
submission = pd.DataFrame({
    'id': sub_id,
    'price': y_test_pred
})
submission.to_csv('submission.csv', index=False)

print("테스트 세트 예측 완료 및 'submission.csv' 파일로 저장되었습니다.")

테스트 세트 예측 완료 및 'submission.csv' 파일로 저장되었습니다.
